<a href="https://colab.research.google.com/github/mahimashah16/NLP_research-papers/blob/main/AI_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!brew install portaudio
!pip install pyaudio
!pip3 install SpeechRecognition pydub
!pip install punctuator
!pip install transformers==2.2.2#for abstractive
!pip install bert-extractive-summarizer #for extractive
!pip install gensim

In [ ]:
import speech_recognition as sr
import os
# from pyhub import AudioSegment
from pydub.silence import split_on_silence
import os
import nltk
from transformers import pipeline
#from summarizer import Summarizer
from nltk import sent_tokenize
import re

nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
r = sr.Recognizer()
with sr.Microphone() as source:
    audio_data = r.record(source, duration=60)
    print("Recognizing......")
    DOCUMENT = r.recognize_google(audio_data)
    print(DOCUMENT)

In [ ]:
from punctuator import Punctuator
p = Punctuator('/Users/deekshitakamble/Downloads/INTERSPEECH-T-BRNN.pcl')
DOCUMENT = p.punctuate(DOCUMENT)
print(p.punctuate(DOCUMENT))

In [ ]:
x=r'/Users/deekshitakamble/Documents/COMP520/audio.txt'

if os.path.exists(x):
    if os.path.isfile(x):
        print("file is existed")
        with open('audio.txt', 'w') as f:
            f.write(DOCUMENT)
            f.close()
    elif os.path.isdir(x):
        print("directory is existed")

else:
    print("file is not existed and creating a new file")
    with open('audio.txt', 'w') as f:
        f.write(DOCUMENT)
        f.close()

In [ ]:
def read_text(path_to_file):
  f = open(path_to_file, 'r', encoding='utf8')
  text = f.read()
  f.close()
  return text

In [ ]:
def preprocess_text(text):
    text = text.strip()
    text = text.replace('\n', ' ').replace('\r', '')
    text = re.sub('\.+', '.', text)
    text = text.replace('«', '"').replace('»', '"')
    text = re.sub('[\.\?\!\;]"', '"', text)
    text = re.sub('[\.\?\!]\;"', ';', text)
    text = re.sub(' +', ' ', text)
    text = re.sub('\t+', ' ', text)
    return text


def smart_tokenization(text):
    sentences = sent_tokenize(text)
    res = []
    cur_text = ''
    for sent in sentences:
        #2000 - max count of symbols for abstractive model
        if len(cur_text + sent) > 2000:
            res.append(cur_text)
            cur_text = sent
        else:
            cur_text += sent
    if cur_text != '':
      res.append(cur_text)
    return res

In [ ]:
class AbstractiveSummarizer():
    def __init__(self):
        self.model = pipeline('summarization')

    def generate_summary(self, text):
        text = preprocess_text(text)
        parts = smart_tokenization(text)
        res = []
        for part in parts:
            res.append(self.model(part)[0]['summary_text'])
        return ' '.join(res)


class ExtractiveSummarizer():
    def __init__(self):
        self.model = Summarizer()

    def generate_summary(self, text):
        text = preprocess_text(text)
        return self.model(text)

In [ ]:
from summarizer import Summarizer
summarizer = AbstractiveSummarizer()

In [ ]:
TEXT_TO_SUMMARIZE = read_text('/Users/deekshitakamble/Documents/COMP520/audio.txt')
summarizer.generate_summary(TEXT_TO_SUMMARIZE) #takes about 40 seconds

In [ ]:
from summarizer import Summarizer
summarizer = ExtractiveSummarizer()

In [ ]:
TEXT_TO_SUMMARIZE = read_text('/Users/deekshitakamble/Documents/COMP520/audio.txt')
summarizer.generate_summary(TEXT_TO_SUMMARIZE) #takes about 40 seconds